# ML and AI Inference: Deploying Models to Production

**The Scenario:** You've trained defect prediction models with AutoML and MLflow. Now leadership needs these models in production - scoring thousands of devices in batch, monitoring sensors in real-time streams, and serving predictions via APIs.

This notebook shows you how to deploy and use your models for:
- **Batch Inference** - Score large datasets all at once
- **Streaming Inference** - Real-time predictions on streaming data
- **Model Serving APIs** - REST endpoints for applications
- **AI Query** - SQL-based inference

## What You'll Learn

✅ Load models from MLflow for batch predictions  
✅ Apply models to streaming data with Structured Streaming  
✅ Deploy models as REST API endpoints  
✅ Use AI Query for SQL-based inference  

---

## Prerequisites

Before starting, you should have:
- ✅ Completed Notebook 4 (AutoML) **or** Notebook 5 (MLflow)
- ✅ At least one model registered in Unity Catalog
- ✅ Model tagged with an alias (e.g., "Champion")

---

**References:**
- [Model Inference Overview](https://docs.databricks.com/aws/en/machine-learning/model-inference/)
- [Deep Learning Model Inference](https://docs.databricks.com/aws/en/machine-learning/model-inference/dl-model-inference)
- [MLflow End-to-End Example](https://docs.databricks.com/aws/en/notebooks/source/mlflow/mlflow-classic-ml-e2e-mlflow-3.html)


In [0]:
%pip install mlflow
%restart_python

In [0]:
# Configuration
import re

catalog = "dwx_express_insights_platform_dev_working"
source_schema = "db_crash_course"  # Source schema (shared, read from here)
username = spark.sql("SELECT current_user()").collect()[0][0]
username_base = username.split('@')[0]  # Extract username before @ symbol
target_schema = re.sub(r'[^a-zA-Z0-9_]', '_', username_base)  # Replace special chars with _

# Your registered model name from AutoML or MLflow notebook
# Model is unique because it's in your personal schema
model_name = f"{catalog}.{target_schema}.iot_defect_predictor"

# Create target schema if it doesn't exist
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{target_schema}")

print(f"✅ Using catalog: {catalog}")
print(f"📖 Reading from schema: {source_schema} (shared)")
print(f"✍️  Writing to schema: {target_schema} (your personal schema)")
print(f"🤖 Model: {model_name}")

## 1. Batch Inference: Score Large Datasets

**Use Case:** You need to score all 400,000 sensor readings to identify which devices are at risk of defects.

**Batch inference** processes large datasets all at once - perfect for:
- Daily/weekly scoring runs
- Historical analysis
- Bulk predictions

### Method 1: Load Model and Apply with PySpark

This is the most common approach for large-scale batch inference.


In [0]:
import mlflow
import mlflow.pyfunc

# Load the Champion model
model_uri = f"models:/{model_name}@Champion"
print(f"Loading model from: {model_uri}")

# Load model as a Spark UDF (User Defined Function)
# This lets us apply the model to Spark DataFrames at scale
predict_udf = mlflow.pyfunc.spark_udf(
    spark, 
    model_uri=model_uri,
    result_type="double"
)

print("✅ Model loaded as Spark UDF")


In [0]:
# Load sensor data for scoring from shared schema
sensor_data = spark.table(f"{catalog}.{source_schema}.sensor_bronze")

# Apply model to make predictions
# The UDF applies the model to each row in parallel across the cluster
from pyspark.sql.functions import when, col

predictions_df = sensor_data.withColumn(
    "defect_probability",
    predict_udf(
        "airflow_rate",
        "rotation_speed",
        "air_pressure",
        "temperature",
        "delay",
        "density"
    )
)

# Add a binary prediction (threshold at 0.5)
predictions_df = predictions_df.withColumn(
    "predicted_defect",
    when(col("defect_probability") > 0.5, 1).otherwise(0)
)

# Show high-risk devices
print("🔴 High-Risk Devices (predicted defects):")
predictions_df.filter("predicted_defect = 1") \
    .select("device_id", "timestamp", "temperature", "rotation_speed", "defect_probability") \
    .orderBy(col("defect_probability").desc()) \
    .limit(20) \
    .display()


In [0]:
# Save predictions to YOUR personal schema for downstream use
predictions_table = f"{catalog}.{target_schema}.sensor_predictions_batch"

predictions_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(predictions_table)

print(f"✅ Predictions saved to: {predictions_table}")
print(f"   Total predictions: {predictions_df.count():,}")
print(f"   Predicted defects: {predictions_df.filter('predicted_defect = 1').count():,}")

## 2. Streaming Inference: Real-Time Monitoring

**Use Case:** New sensor readings arrive continuously from aircraft. You want to score them in real-time and alert on predicted defects within seconds.

**Structured Streaming** + MLflow lets you apply models to streaming data.

### How Streaming Inference Works

```
Sensor Data Stream → Load Model UDF → Apply Predictions → Write to Table → Alert if Defect
```


In [0]:
# Create a streaming DataFrame (simulated - reads from table as stream)
# In production, this would connect to Kafka, Event Hub, or Kinesis
streaming_sensor_data = spark.readStream \
    .format("delta") \
    .table(f"{catalog}.{source_schema}.sensor_bronze")

# Apply the model UDF to streaming data
streaming_predictions = streaming_sensor_data.withColumn(
    "defect_probability",
    predict_udf(
        "airflow_rate",
        "rotation_speed",
        "air_pressure",
        "temperature",
        "delay",
        "density"
    )
).withColumn(
    "predicted_defect",
    when(col("defect_probability") > 0.5, 1).otherwise(0)
)

print("✅ Created streaming pipeline with model predictions")
print("   Each new sensor reading will be scored in real-time")
print("   To deploy a real time streaming model, write the results to a table or an external system and run that stream in a continuous job. For testing, you could .display() the results")

## 3. Model Serving: REST API Endpoints

**Use Case:** Your operations dashboard needs to call the model via API to show real-time risk scores. External applications need to integrate with your predictions.

**Model Serving** deploys your model as a REST API endpoint.

### Step 1: Create a Serving Endpoint (UI)

**In the Databricks UI:**

1. Click **AI/ML** in the sidebar
2. Click **Serving**
3. Click **Create Serving Endpoint**
4. Configure:
   - **Name**: `{username}-iot-defect-predictor`
   - **Model**: Select your registered model (`iot_defect_predictor`) from Unity Catalog
   - **Model version**: Select the most recent version
   - **Compute size**: Small (for testing)
   - **Scale to zero**: Enabled (saves costs when idle)
5. Click **Create**

**Wait ~10 minutes** for the endpoint to deploy.

### What Just Happened?

- Databricks deployed your model as a managed REST API
- Auto-scaling handles traffic spikes
- Monitoring tracks latency and throughput
- Authentication is handled automatically


### Step 2: Test the API and Use AI Query

Once deployed, you can call the endpoint from Python, SQL (`ai_query`), or any HTTP client.

**⚠️ Note:** To use `ai_query` in SQL, your model must be deployed as a Model Serving endpoint first.


In [0]:
%sql
-- Example: Using ai_query for SQL-based inference
-- (Uncomment after deploying model serving endpoint)

-- SELECT 
--     device_id,
--     timestamp,
--     temperature,
--     rotation_speed,
--     ai_query(
--         'iot-defect-predictor', -- Make sure to use your endpoint's name here!
--         airflow_rate,
--         rotation_speed,
--         air_pressure,
--         temperature,
--         delay,
--         density
--     ) as defect_probability
-- FROM dwx_express_insights_platform_dev_working.db_crash_coursesensor_bronze
-- ORDER BY defect_probability DESC
-- LIMIT 20;

SELECT '✅ AI Query syntax ready - uncomment after deploying serving endpoint' as status


## Summary

Congratulations! You've learned how to deploy ML models for production inference.

### What You Accomplished

✅ **Batch inference** - Scored large datasets with Spark UDFs  
✅ **Streaming inference** - Applied models to real-time data streams  
✅ **Model serving** - Learned how to deploy REST API endpoints  
✅ **AI Query** - Saw SQL-based inference examples  

### Key Takeaways

1. **Multiple deployment patterns** - Choose based on use case (batch, streaming, API)
2. **Spark UDFs** - Scale ML inference across distributed data
3. **Model Serving** - Managed REST APIs with auto-scaling
4. **AI Query** - SQL-native inference for dashboards and BI tools
5. **Same model, different patterns** - One trained model, multiple deployment options

### Comparing Inference Methods

| Method | Best For | Latency | Scale | Complexity |
|--------|----------|---------|-------|------------|
| **Batch (Spark UDF)** | Daily scoring, large datasets | Minutes | Millions of rows | Low |
| **Streaming (Spark UDF)** | Continuous monitoring | Seconds | Thousands/sec | Medium |
| **API Serving** | Interactive apps, low latency | Milliseconds | Hundreds/sec | Medium |
| **AI Query (SQL)** | Dashboards, SQL tools | Milliseconds | Hundreds/sec | Low |

### What's Next?

Your aircraft IoT ML system is now complete:
- ✅ Data discovered and transformed (Day 1)
- ✅ Dashboards and Genie spaces built (Day 1 & 2)
- ✅ Models trained with AutoML and MLflow (Day 2)
- ✅ Models deployed for inference (this notebook)

**Day 3** covers production operations:
- CI/CD for automated deployment
- Performance tuning
- Monitoring and governance

**Leadership's deadline is in sight!** 🎯


## Try This Out

Want to go further? Try these exercises:

### 1. Try an AI Query with an LLM!
```
SELECT
  messages, 
  ai_query(
    'databricks-gpt-oss-20b',
    CONCAT('good or bad customer interaction?', to_json(messages))  
  ) AS response
FROM samples.wanderbricks.customer_support_logs  
LIMIT 5;
```

### 2. Schedule Batch Inference Job

Automate daily scoring:
1. Create a new notebook with just the batch inference code
2. In **Workflows**, create a new job
3. Add your inference notebook as a task
4. Schedule for 2 AM daily
5. Add notification on completion

**Result:** Fresh predictions every morning before leadership arrives!

---

### 3. Build a Real-Time Risk Dashboard

Create a dashboard that shows:
- Current devices with high defect risk (from predictions)
- Risk trends over time
- Alert counts by factory
- Model prediction distribution

Use the `sensor_predictions_batch` table as the data source!

---

### 4. Deploy Your Model Serving Endpoint

Follow the steps in section 3 to:
1. Deploy your model as a serving endpoint
2. Test the API with Python
3. Try `ai_query` from SQL
4. Add the endpoint to your Genie space

---

### 5. Create an Alert System

Build a streaming query that writes high-risk predictions to a separate table:

```python
# Filter for high-risk predictions
alerts = streaming_predictions.filter("defect_probability > 0.8")

# Write to alerts table
alerts.writeStream \
    .format("delta") \
    .outputMode("append") \
    .toTable(f"{catalog}.{schema}.defect_alerts")
```

Then create an Alert from the **Alerts** tab on the left!

---

### 6. Add a predictions step to the Declarative ETL Pipeline or job from Day 1 
Add your ML model's predictions to the scheduled job from yesterday using one of the methods from this notebook

---

**Additional Resources:**
- [Model Inference Guide](https://docs.databricks.com/aws/en/machine-learning/model-inference/)
- [Model Serving](https://docs.databricks.com/aws/en/machine-learning/model-serving/)
- [Structured Streaming + ML](https://docs.databricks.com/aws/en/structured-streaming/apply-ml-models.html)

**🎉 Congratulations!** You've completed the full ML lifecycle: data → training → deployment. You're ready for Day 3!
